# Issue Valume Predictions for Comparable Markets

This notebook tries to build models and forecast issue volumes for all components in TQ,AQ and TS for 6 countries recognized as comparable markets 

In [1]:
import time
begin = time.ctime()
begin

'Fri Mar 25 03:29:10 2022'

In [4]:
import numpy as np
import pandas as pd
pd.set_option('max.rows',100)

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from datetime import datetime

#sklearn
from sklearn.metrics import mean_squared_error

#Time Series
from pylab import rcParams
import statsmodels.api as sm

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss

from scipy.stats import boxcox

from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing
#Params used

ip_path ='./Input/'
op_path ='./Output/'

rcParams['figure.figsize'] = 12, 8


#We have data from Sept 2015, Total months = 69
#Train data length
train_len = 64
#Data to be forecasted or test data length
fc_mths = 5

#Starting month
start_mth = 4 #4-starts from JaN 2016, 28 - starts from Jan 2018

exclude_mons = [] #['01-09-2015','01-10-2015']
exclude_mon_nums=[] #[20150901,20151001]

#Create an empty data frame with our date range with values filled as nan
start = pd.datetime(2015,9,1) #Date from which we have data
end = pd.datetime(2021,5,1) #Current Month  
index = pd.date_range(start, end,freq='MS')

# Create an empty data frame with our date range with values filled as nan
empty_ts = pd.DataFrame(index=index)
empty_ts['Issues'] = np.nan
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

from pmdarima.arima import auto_arima

In [5]:
#Params used

ip_path ='./Input/'
op_path ='./Output/'

rcParams['figure.figsize'] = 12, 8


#We have data from Sept 2015, Total months = 69
#Train data length
train_len = 64
#Data to be forecasted or test data length
fc_mths = 5

#Starting month
start_mth = 4 #4-starts from JaN 2016, 28 - starts from Jan 2018

exclude_mons = [] #['01-09-2015','01-10-2015']
exclude_mon_nums=[] #[20150901,20151001]

#Create an empty data frame with our date range with values filled as nan
start = pd.datetime(2015,9,1) #Date from which we have data
end = pd.datetime(2021,5,1) #Current Month  
index = pd.date_range(start, end,freq='MS')

# Create an empty data frame with our date range with values filled as nan
empty_ts = pd.DataFrame(index=index)
empty_ts['Issues'] = np.nan

In [6]:
# Function to Print null counts with counts greater than threshold.
#If threshold is not specified, pritns all null counts
def get_null_cnts(df_, thresh = None):
    null_cnts = df_.isnull().sum()
    null_cnts = null_cnts.sort_values(ascending=False)
    if thresh == None :
        return null_cnts
    else :
        return null_cnts[null_cnts > thresh]

In [7]:
# Function to Print null percentages with percentage greater than threshold.
#If threshold is not specified, pritns all null percentages
def get_null_perc(df_, thresh = None):
    null_perc = round(100 * df_.isnull().sum()/len(df_), 2)
    null_perc = null_perc.sort_values(ascending=False)
    if thresh == None :
        return null_perc
    else :
        return null_perc[null_perc > thresh]

In [8]:
#Function to get the group as test or control
def get_group(x):
    if x in test_cids:
        return 'Test'
    else:
        return 'Control'

In [9]:
#Date handling functions

def convert_dt_str_to_mth(dt_str_):
    dt_ = datetime.strptime(dt_str_,'%d-%m-%Y %H:%M')
    return datetime.strftime(dt_,'%B%Y')

#Format 2020-09-01
def convert_dt_str_to_mth2(dt_str_):
    dt_ = datetime.strptime(dt_str_,'%Y-%m-%d')
    return datetime.strftime(dt_,'%B%Y')


#Date Format:20180701
def convert_dt_int_to_mth(dt_int_):
    dt_ = datetime.strptime(str(dt_int_),'%Y%m%d')
    return datetime.strftime(dt_,'%B%Y')

#Date Format:20180701
def convert_dt_int_to_dt(dt_int_):
    return datetime.strptime(str(dt_int_),'%Y%m%d')

def convert_mth_to_dt(mth_):
    return datetime.strptime(mth_,'%B%Y')

def add_months(dt_, num_):
    return dt_ + relativedelta(months=num_)

def convert_dt_to_mth(dt_):
    return datetime.strftime(dt_,'%B%Y')

def convert_mth_to_date_key(mth_):
    return datetime.strftime(convert_mth_to_dt(mth_),'%Y%m%d')

def get_mth_range(dt_):
    start = add_months(dt_,- (pre_post_months))
    end = add_months(dt_,(pre_post_months))
    dt_range = pd.date_range(start,end,freq='MS')
    mth_range = dt_range.map(convert_dt_to_mth)
    mth_range
    return mth_range

#This function is to get all previous months engaged prior to passed month. To derive if the customer is new in curent month
def get_prev_mths(mth_):
    return eng_dt_range[:eng_dt_range.index(mth_)]

In [10]:
me_df_orig = pd.read_excel('./Input/ME_Analysis.xlsx',sheet_name='Sheet1')

In [11]:
me_df_orig.shape

(29882, 12)

In [12]:
me_df_orig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29882 entries, 0 to 29881
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   CalendarMonthStart    29882 non-null  int64 
 1   Editorial Language    29882 non-null  object
 2   Issues                29882 non-null  int64 
 3   EditorialCountryName  29882 non-null  object
 4   ComponentName         29882 non-null  object
 5   QueueCategory         29882 non-null  object
 6   QueueCategory1        29882 non-null  object
 7   QueueCategory2        12680 non-null  object
 8   Country&Lang          29882 non-null  object
 9   Country&Lang(T/F)     147 non-null    object
 10  Only Country          1103 non-null   object
 11  Only Lang             16017 non-null  object
dtypes: int64(2), object(10)
memory usage: 2.7+ MB


In [13]:
me_df_orig.head(2)

,CalendarMonthStart,Editorial Language,Issues,EditorialCountryName,ComponentName,QueueCategory,QueueCategory1,QueueCategory2,Country&Lang,Country&Lang(T/F),Only Country,Only Lang
0,20160101,(blank),1,Australia,Appeals,NQP,AQ,NaN,Australia(blank),NaN,NaN,NaN
1,20160101,(blank),59,China,Appeals,NQP,AQ,NaN,China(blank),NaN,NaN,NaN


In [14]:
me_df_orig['Editorial Language'].value_counts()

English                11792
(blank)                 8465
Spanish                 1917
German                  1897
French                  1521
Traditional Chinese     1038
Dutch                    749
Portuguese               747
Italian                  639
Swedish                  338
Danish                   298
Norwegian                220
Finnish                  182
Hungarian                 11
Polish                    11
Japanese                   9
Romanian                   9
Slovak                     6
Croatian                   5
Catalan                    5
Czech                      5
Greek                      4
Slovenian                  4
Estonian                   3
Bulgarian                  3
Albanian                   1
Turkish                    1
Lithuanian                 1
Latvian                    1
Name: Editorial Language, dtype: int64

In [15]:
#Make a copy before doing any manipulations
me_df = me_df_orig.copy()

In [16]:
#Convert Month Start column to data format
me_df['Month'] = me_df.CalendarMonthStart.map(convert_dt_int_to_dt)

In [18]:
cats_df = me_df[['ComponentName','QueueCategory1','QueueCategory2']].drop_duplicates()

In [20]:
#Components mapped to Tscore
ts_core_comps = list(cats_df[(cats_df.QueueCategory1=='TS') & (cats_df.QueueCategory2=='Tscore')].ComponentName)
ts_core_comps

['Advertiser API',
 'Analytics',
 'BI/Reporting',
 'Campaign Mgmt',
 'Editorial',
 'Microsoft Advertising Editor',
 'Optimization',
 'Product Ads/Shopping',
 'Syndication/Publisher API',
 'Tier 3',
 'UCMA',
 'UCMT',
 'UCMB',
 'UCMC',
 'UCMBA']

In [21]:
#Number of rows mapped to NaN group for TS
me_df[ ((me_df.QueueCategory1=='TS') & (me_df.QueueCategory2.isnull()))].shape

(430, 13)

#### Very few records for nulls in QueueCategory2. We can ignore them safely

In [22]:
#Drop the TS components data not mapped to any group 
me_df = me_df[ ~((me_df.QueueCategory1=='TS') & (me_df.QueueCategory2.isnull()))]

In [23]:
#Introduce a new column with Component Names grouped for TS and as is for AQ and TQ
me_df['Component'] = me_df.ComponentName.replace(ts_core_comps,'Tscore')

In [24]:
#Count of rows by component
me_df.Component.value_counts()

Tscore                        8664
Phishing/Techscam             5808
Appeals                       4019
Complaints                    3480
Accounts & Billing            1841
Delivery                      1679
Supply TQ Investigation        948
Malware/Bundling/Downloads     820
Policy Questions               801
Enablements                    404
Matching                       314
Bing Places                    291
Bing Web Master Tools          205
SEAM                           178
Name: Component, dtype: int64

In [25]:
me_df.head(2)

,CalendarMonthStart,Editorial Language,Issues,EditorialCountryName,ComponentName,QueueCategory,QueueCategory1,QueueCategory2,Country&Lang,Country&Lang(T/F),Only Country,Only Lang,Month,Component
0,20160101,(blank),1,Australia,Appeals,NQP,AQ,NaN,Australia(blank),NaN,NaN,NaN,2016-01-01,Appeals
1,20160101,(blank),59,China,Appeals,NQP,AQ,NaN,China(blank),NaN,NaN,NaN,2016-01-01,Appeals


In [26]:
#Rename EditorialCountryName column to Country 
me_df = me_df.rename(columns={'EditorialCountryName':'Country'})

me_df = me_df.rename(columns={'Editorial Language':'Language'})

In [27]:
me_df

,CalendarMonthStart,Language,Issues,Country,ComponentName,QueueCategory,QueueCategory1,QueueCategory2,Country&Lang,Country&Lang(T/F),Only Country,Only Lang,Month,Component
0,20160101,(blank),1,Australia,Appeals,NQP,AQ,NaN,Australia(blank),NaN,NaN,NaN,2016-01-01,Appeals
1,20160101,(blank),59,China,Appeals,NQP,AQ,NaN,China(blank),NaN,NaN,NaN,2016-01-01,Appeals
2,20160101,(blank),5,China,Complaints,NQP,AQ,NaN,China(blank),NaN,NaN,NaN,2016-01-01,Complaints
3,20160101,(blank),1,China,Policy Questions,NQP,AQ,NaN,China(blank),NaN,NaN,NaN,2016-01-01,Policy Questions
4,20160101,(blank),1,France,Appeals,NQP,AQ,NaN,France(blank),NaN,NaN,NaN,2016-01-01,Appeals
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29877,20210601,Croatian,2,Croatia,Appeals,NQP,AQ,NaN,CroatiaCroatian,CroatiaCroatian,Croatia,Croatian,2021-06-01,Appeals
29878,20210601,Croatian,1,Croatia,Complaints,NQP,AQ,NaN,CroatiaCroatian,CroatiaCroatian,Croatia,Croatian,2021-06-01,Complaints
29879,20210401,Bulgarian,1,Bulgaria,Appeals,NQP,AQ,NaN,BulgariaBulgarian,BulgariaBulgarian,Bulgaria,Bulgarian,2021-04-01,Appeals
29880,20210601,Bulgarian,2,Bulgaria,Appeals,NQP,AQ,NaN,BulgariaBulgarian,BulgariaBulgarian,Bulgaria,Bulgarian,2021-06-01,Appeals


In [28]:
#Select the columns that we are interested in
me_df = me_df[['Month','Language','Country','Component','Issues']]

me_df['Language'].replace(to_replace ="(blank)",
                 value ="English", inplace=True)
me_df


,Month,Language,Country,Component,Issues
0,2016-01-01,English,Australia,Appeals,1
1,2016-01-01,English,China,Appeals,59
2,2016-01-01,English,China,Complaints,5
3,2016-01-01,English,China,Policy Questions,1
4,2016-01-01,English,France,Appeals,1
...,...,...,...,...,...
29877,2021-06-01,Croatian,Croatia,Appeals,2
29878,2021-06-01,Croatian,Croatia,Complaints,1
29879,2021-04-01,Bulgarian,Bulgaria,Appeals,1
29880,2021-06-01,Bulgarian,Bulgaria,Appeals,2


In [29]:
me_df.Language.value_counts()

English                19841
Spanish                 1911
German                  1896
French                  1521
Traditional Chinese     1036
Dutch                    749
Portuguese               744
Italian                  637
Swedish                  338
Danish                   298
Norwegian                220
Finnish                  182
Hungarian                 11
Polish                    11
Japanese                   9
Romanian                   9
Slovak                     6
Croatian                   5
Czech                      5
Catalan                    5
Greek                      4
Slovenian                  4
Estonian                   3
Bulgarian                  3
Albanian                   1
Turkish                    1
Lithuanian                 1
Latvian                    1
Name: Language, dtype: int64

In [30]:
print('Number of Unique Languages:', len(me_df.Language.unique()), 'Number Of Components', len(me_df.Component.unique()))

Number of Unique Languages: 28 Number Of Components 14


In [31]:
#Comparable Market countries list are the ones we consider
#cons_cons = ['Denmark', 'Finland', 'France', 'Norway', 'Spain', 'Sweden']
#cons_cons = ['Danish','Swedish','French','Norwegian','Sami','Spanish','Swedish']
cons_cons_c = ['Denmark', 'Finland', 'France', 'Norway', 'Spain', 'Sweden']
#cons_cons = ['Danish','Swedish']
comp_me_df = me_df[me_df.Country.isin(cons_cons_c)]
#comp_me_df = me_df[me_df.Language.isin(cons_cons)]

In [32]:
#Counts by country
comp_me_df.Language.value_counts()

English                1893
French                  843
Spanish                 633
Swedish                 300
Danish                  277
Norwegian               204
Finnish                 175
German                   68
Italian                  27
Dutch                    21
Portuguese               20
Traditional Chinese      11
Name: Language, dtype: int64

In [33]:
#Number of Country Component Combinations to consider
con_comp = comp_me_df[['Language','Component']].drop_duplicates().sort_values(['Language','Component'])
con_comp = comp_me_df[['Language','Component']].drop_duplicates().sort_values(['Language','Component'])
print('Number of combinations:', len(con_comp) )

#First and last month for which we have data for a combination
con_comp['FirstMonth'] = np.nan
con_comp['LastMonth'] = np.nan
#Number of months we have data for a combination
con_comp['Total'] = 0
#Add a column for number of missing entries in between min and max of available months
con_comp['Missing'] = 0
#Continuously for how many months we have data from the last month backwards
con_comp['LastCont'] = 0
#Average Isssues per month for a combinations
con_comp['AvgIssues'] = 0

Number of combinations: 112


In [35]:
con_comp

,Language,Component,FirstMonth,LastMonth,Total,Missing,LastCont,AvgIssues
17580,Danish,Accounts & Billing,NaN,NaN,0,0,0,0
18,Danish,Appeals,NaN,NaN,0,0,0,0
19,Danish,Complaints,NaN,NaN,0,0,0,0
17855,Danish,Delivery,NaN,NaN,0,0,0,0
4544,Danish,Enablements,NaN,NaN,0,0,0,0
...,...,...,...,...,...,...,...,...
6877,Traditional Chinese,Appeals,NaN,NaN,0,0,0,0
4125,Traditional Chinese,Complaints,NaN,NaN,0,0,0,0
23133,Traditional Chinese,Delivery,NaN,NaN,0,0,0,0
13754,Traditional Chinese,Phishing/Techscam,NaN,NaN,0,0,0,0


#### Out of 84Country/Component combinations, we have data for 75 combinations 

In [36]:
comp_me_df.head(2)

,Month,Language,Country,Component,Issues
4,2016-01-01,English,France,Appeals,1
18,2016-01-01,Danish,Denmark,Appeals,10


In [42]:
con_comp.head()

,Language,Component,FirstMonth,LastMonth,Total,Missing,LastCont,AvgIssues
17580,Danish,Accounts & Billing,NaN,NaN,0,0,0,0
18,Danish,Appeals,NaN,NaN,0,0,0,0
19,Danish,Complaints,NaN,NaN,0,0,0,0
17855,Danish,Delivery,NaN,NaN,0,0,0,0
4544,Danish,Enablements,NaN,NaN,0,0,0,0


In [49]:
for i in con_comp.itertuples():
    print((i))

Pandas(Index=17580, Language='Danish', Component='Accounts & Billing', FirstMonth=nan, LastMonth=nan, Total=0, Missing=0, LastCont=0, AvgIssues=0)
Pandas(Index=18, Language='Danish', Component='Appeals', FirstMonth=nan, LastMonth=nan, Total=0, Missing=0, LastCont=0, AvgIssues=0)
Pandas(Index=19, Language='Danish', Component='Complaints', FirstMonth=nan, LastMonth=nan, Total=0, Missing=0, LastCont=0, AvgIssues=0)
Pandas(Index=17855, Language='Danish', Component='Delivery', FirstMonth=nan, LastMonth=nan, Total=0, Missing=0, LastCont=0, AvgIssues=0)
Pandas(Index=4544, Language='Danish', Component='Enablements', FirstMonth=nan, LastMonth=nan, Total=0, Missing=0, LastCont=0, AvgIssues=0)
Pandas(Index=5441, Language='Danish', Component='Matching', FirstMonth=nan, LastMonth=nan, Total=0, Missing=0, LastCont=0, AvgIssues=0)
Pandas(Index=8954, Language='Danish', Component='Phishing/Techscam', FirstMonth=nan, LastMonth=nan, Total=0, Missing=0, LastCont=0, AvgIssues=0)
Pandas(Index=20, Language='

In [31]:
#Check the data availability for the combinations
for row in con_comp.itertuples():
    con, comp = row[1], row[2]
    temp_df = me_df[(me_df.Language == con) & (me_df.Component == comp)].groupby('Month')['Issues'].sum().reset_index()
    
    #First and Last month for which we have data for 
    first_mon = temp_df.Month.min()
    last_mon = temp_df.Month.max()
    
    #Month Range between first and last months 
    mth_range = pd.date_range( first_mon, last_mon, freq='MS')
    
    #Missing months in the range
    missing_mths = mth_range.difference(temp_df.Month)
    
    if len(missing_mths) > 0:
        #If data missing for a month or more, how many months data do we have continuosly from last month
        last_cont_mths = pd.date_range(max(missing_mths), last_mon, freq='MS')
    else:
        last_cont_mths = mth_range
    
    con_comp.loc[(con_comp.Language==con) & (con_comp.Component==comp),'FirstMonth'] = convert_dt_to_mth(first_mon)
    con_comp.loc[(con_comp.Language==con) & (con_comp.Component==comp),'LastMonth'] = convert_dt_to_mth(last_mon)
    con_comp.loc[(con_comp.Language==con) & (con_comp.Component==comp),'Total'] = len(mth_range)
    con_comp.loc[(con_comp.Language==con) & (con_comp.Component==comp),'Missing'] = len(missing_mths)
    con_comp.loc[(con_comp.Language==con) & (con_comp.Component==comp),'LastCont'] = len(last_cont_mths)-1 # Minus one
    
    #Average Issues
    con_comp.loc[(con_comp.Language==con) & (con_comp.Component==comp),'AvgIssues'] = temp_df.Issues.mean()

In [32]:
con_comp['MissingPerc'] = round((con_comp.Missing/con_comp.Total)*100)
con_comp['DataAvl'] = con_comp.Total - con_comp.Missing

In [33]:
con_comp.head(2)

,Language,Component,FirstMonth,LastMonth,Total,Missing,LastCont,AvgIssues,MissingPerc,DataAvl
17580,Danish,Accounts & Billing,November2016,January2021,51,21,7,1.266667,41.0,30
18,Danish,Appeals,January2016,June2021,66,0,65,15.136364,0.0,66


## Filter out combinations without significant data

In [34]:
#Ignore combinations not having more than 15 months of data
cons_con_comp = con_comp[con_comp.DataAvl > 15]
cons_con_comp.shape

(82, 10)

#### 27 combinations are gone if we consider combinations with at least 15 months of data availability

In [35]:
#Ignored combinations are
con_comp[~(con_comp.DataAvl > 15)]

,Language,Component,FirstMonth,LastMonth,Total,Missing,LastCont,AvgIssues,MissingPerc,DataAvl
17855,Danish,Delivery,March2017,May2020,39,26,1,1.153846,67.0,13
4544,Danish,Enablements,April2019,March2021,24,12,2,1.833333,50.0,12
5441,Danish,Matching,October2019,January2021,16,12,1,1.250000,75.0,4
20,Danish,Policy Questions,January2016,December2020,60,52,1,1.375000,87.0,8
12195,Danish,Supply TQ Investigation,June2018,January2021,32,19,1,1.153846,59.0,13
17756,Finnish,Accounts & Billing,March2017,October2020,44,36,1,1.125000,82.0,8
20961,Finnish,Bing Web Master Tools,June2018,June2018,1,0,0,1.000000,0.0,1
22713,Finnish,Delivery,March2019,June2021,28,25,1,1.000000,89.0,3
5953,Finnish,Matching,January2020,July2020,7,5,1,1.000000,71.0,2
63,Finnish,Policy Questions,January2016,April2019,40,35,1,1.600000,88.0,5


In [36]:
#This can be configred based on the requirement
recent_months = ['June2021','May2021','April2021']
display('Ignored Combinations are:', cons_con_comp[~cons_con_comp.LastMonth.isin(recent_months)])
cons_con_comp = cons_con_comp[cons_con_comp.LastMonth.isin(recent_months)]
cons_con_comp.shape

'Ignored Combinations are:'

,Language,Component,FirstMonth,LastMonth,Total,Missing,LastCont,AvgIssues,MissingPerc,DataAvl
17580,Danish,Accounts & Billing,November2016,January2021,51,21,7,1.266667,41.0,30
8954,Danish,Phishing/Techscam,January2016,June2020,54,30,1,1.958333,56.0,24
9023,Finnish,Phishing/Techscam,February2016,May2019,40,22,1,1.333333,55.0,18
9048,French,Malware/Bundling/Downloads,March2016,January2021,59,34,1,1.400000,58.0,25
70,French,Policy Questions,January2016,February2021,62,18,2,2.659091,29.0,44
13638,German,Malware/Bundling/Downloads,February2016,October2020,57,29,1,1.500000,51.0,28
9384,Norwegian,Phishing/Techscam,June2016,May2019,36,20,1,1.812500,56.0,16
9050,Spanish,Malware/Bundling/Downloads,March2016,March2021,61,29,1,2.125000,48.0,32
17774,Swedish,Accounts & Billing,March2017,July2019,29,12,2,1.529412,41.0,17
9292,Swedish,Phishing/Techscam,May2016,November2020,55,31,1,2.291667,56.0,24


(71, 10)

#### Another 5 combinations are gone if we do not 

In [37]:
cons_con_comp

,Language,Component,FirstMonth,LastMonth,Total,Missing,LastCont,AvgIssues,MissingPerc,DataAvl
18,Danish,Appeals,January2016,June2021,66,0,65,15.136364,0.0,66
19,Danish,Complaints,January2016,May2021,65,1,53,21.140625,2.0,64
16671,Danish,Tscore,January2016,June2021,66,31,2,1.457143,47.0,35
6893,Dutch,Appeals,January2016,June2021,66,0,65,46.484848,0.0,66
1174,Dutch,Complaints,January2016,June2021,66,0,65,50.878788,0.0,66
8422,Dutch,Enablements,April2019,June2021,27,3,19,5.041667,11.0,24
24716,Dutch,Tscore,February2016,June2021,65,11,13,7.000000,17.0,54
16697,English,Accounts & Billing,January2016,June2021,66,0,65,486.712121,0.0,66
4,English,Appeals,January2016,June2021,66,0,65,1626.045455,0.0,66
16717,English,Bing Places,January2016,June2021,66,0,65,755.712121,0.0,66


#### By looking at the remaining combination's data, it seems to be dafe to impute missing month's data with zero

## Build models

In [38]:
#Number of combinations to build models for
cons_con_comp.shape

(71, 10)

In [39]:
#Function to calculate error and prepare a data frame
def calc_error(method, act, pred):
    global cons_err_df
 
    #Root Mean Square Error
    rmse = np.sqrt(mean_squared_error(act, pred)).round(2)
    mae = np.mean(abs(act-pred))

#     cons_res_df['Actual_'+method] = results_df_['Issues'] #Not required for every method. Initiate it only once
    cons_res_df['Forecast_'+method] = pred
    
 
    #Mean Absolute Percentage error
    mape = np.round(np.mean(np.abs(act-pred)/act)*100,2)
    
    # Over and Under predictions to check the direction of the error
    over_pred, under_pred = [],[]
    for row in zip(act,pred):
        actual = row[0]
        predicted = row[1]
        if predicted > actual:
            over_pred.append([predicted,actual])
        else:
            under_pred.append([predicted,actual])

    
    #This is to avoid shown nan as average error
    if len(over_pred) == 0:
        over_pred_mape =0
    else:
#         over_pred_avg = round(np.mean(over_pred),1)
        preds = [item[0] for item in over_pred]
        acts = [item[1] for item in over_pred]
        over_pred_mape = np.round(np.mean(np.abs(np.array(preds)-np.array(acts))/np.array(acts))*100,2)

    if len(under_pred) == 0:
        under_pred_mape = 0
    else:
#         under_pred_avg = round(np.mean(under_pred),1)
        preds = [item[0] for item in under_pred]
        acts = [item[1] for item in under_pred]
        under_pred_mape = np.round(np.mean(np.abs(np.array(preds)-np.array(acts))/np.array(acts))*100,2)

    over_preds = str(len(over_pred)) + ' - ' + str(over_pred_mape)
    under_preds = str(len(under_pred)) + ' - ' + str(under_pred_mape)

 
    err_df_ = pd.DataFrame({'Method':[method], 'MAPE': [mape], 'RMSE': [rmse], 'MAE':[mae],
                           'Over Predictions':[over_preds], 'Under Predictions':[under_preds]})
    cons_err_df = cons_err_df.append(err_df_)
    
#     print('Method:', method, 'MAPE:', [mape], 'RMSE:', [rmse], 'MAE:',[mae])
    return err_df_

In [40]:
#Plot results of a time series forecasting model
def plot_results(method_name_, ts_data_train, results_df_):
    plt.figure(figsize=(12,4))
    plt.plot(ts_data_train, label='Train')
    plt.plot(results_df_['Issues'], label='Test')
    plt.plot(results_df_['Issues_FC'], label=method_name_+' Forecast')
    plt.legend(loc='best')
    plt.title(method_name_)
    plt.show()

In [41]:
#Functions to run different models

#Simple Exponential Smoothing
def run_simple_exp_smooth(ts_data_train, ts_data_test, smoothing_level_=0.2, optimized_=False):
    model = SimpleExpSmoothing(ts_data_train)
    try:
        model_fit = model.fit(smoothing_level=smoothing_level_, optimized=optimized_)
        if debug:
            print(model_fit.params)
    except ValueError as e:
        print('Value Error while fitting the model SES:', str(e))
        return pd.DataFrame()

    results_df = ts_data_test.to_frame()
    results_df['Issues_FC'] = model_fit.forecast(len(ts_data_test))
    
    return results_df


#Holts method with only trend
def run_holt_trend(ts_data_train, ts_data_test, seasonal_periods_=12 , smoothing_level_=0.2, smoothing_slope_=0.01, optimized_=False):
    
    model = ExponentialSmoothing(np.asarray(ts_data_train) ,seasonal_periods = seasonal_periods_ , trend='additive', seasonal=None)

    try:
        model_fit = model.fit(smoothing_level = smoothing_level_, smoothing_slope = smoothing_slope_, optimized = optimized_)
        if debug:
            print(model_fit.params)
    except ValueError as e:
        print('Value Error while fitting the model Holt with only Trend:', str(e))
        return pd.DataFrame()
    
    results_df = ts_data_test.to_frame()
    results_df['Issues_FC'] = model_fit.forecast(len(ts_data_test))
    
    return results_df

#Holt winters with trend and seasonality
def run_holt_wint(ts_data_train, ts_data_test, seasonal_periods_=12 , trend_='add', seas_='add', optimized_=True, use_boxcox_=None,init_method_=None):
    
    model = ExponentialSmoothing(np.asarray(ts_data_train) ,seasonal_periods=seasonal_periods_ ,trend=trend_, seasonal=seas_, 
                                 initialization_method=init_method_, use_boxcox=use_boxcox_)
    
    try:
        model_fit = model.fit(optimized=optimized_)
        if debug:
            print(model_fit.params)
    except ValueError as e:
        print('Value Error while fitting the model Holt Winters:', str(e))
        return pd.DataFrame()
        
    results_df = ts_data_test.to_frame()
    results_df['Issues_FC'] = model_fit.forecast(len(ts_data_test))
    
    return results_df

#ARIMA
def run_arima(ts_data_train, ts_data_test, is_bc_=False, order_=(1,0,0)):
    
    model = ARIMA(ts_data_train, order=order_)
    try:
        model_fit = model.fit()
        if debug:
            print(model_fit.params)
    except ValueError as e:
        print('Value Error while fitting the model ARIMA:', str(e))
        return pd.DataFrame()
    
    results_df = ts_data_test.to_frame()
    results_df['Issues_FC'] = model_fit.forecast(len(ts_data_test))[0]

    if is_bc_:
        results_df['Issues'] = np.exp(results_df['Issues'])
        results_df['Issues_FC'] = np.exp(results_df['Issues_FC'])

    return results_df


#SARIMAX
def run_sarimax(ts_data_train, ts_data_test, is_bc_=False, order_=(1,1,1), seasonal_order_=(1,1,1,12), is_exog_=False, exog_=None):

    promo_train_, promo_test_ = None, None 
    if is_exog_:
        promo_train_ = exog_[0:len(ts_data_train)+1]
        promo_test_ = exog_[len(ts_data_train):]
        print(ts_data_train.shape, ts_data_test.shape, promo_train_.shape, promo_test_.shape)
    
    model = SARIMAX(ts_data_train, order=order_, seasonal_order=seasonal_order_,exog=promo_train_)

    try:
        model_fit = model.fit()
        if debug:
            print(model_fit.params)
    except ValueError as e:
        print('Value Error while fitting the model SARIMAX:', str(e))
        return pd.DataFrame()
    
    results_df = ts_data_test.to_frame()
    results_df['Issues_FC'] = model_fit.forecast(len(ts_data_test))
    
    if is_bc_:
        results_df['Issues'] = np.exp(results_df['Issues'])
        results_df['Issues_FC'] = np.exp(results_df['Issues_FC'])

    return results_df

def run_auto_arima(ts_data_train, ts_data_test, is_bc_=False, start_p=1, start_q=1,max_d=2, max_p=3, max_q=3,start_P=0, 
                             start_Q=0, max_P=3, max_D=1, max_Q=3, m=12, seasonal=True, trace=True, d=1, D=1, max_order = 5,
                             maxiter = 50,
                             error_action='warn', suppress_warnings=True, stepwise=True, random_state=20, n_fits=100, n_jobs = -1):

    # model = auto_arima(train, trace=True, error_action='ignore', suppress_warnings=True)
    model = auto_arima(ts_data_train,start_p=start_p,start_q=start_q,max_p=max_p, max_d=max_d, max_q=max_q,start_P=start_P, 
                             start_Q=start_Q, max_P=max_P, max_D=max_D, max_Q=max_Q, m=m, seasonal=seasonal, trace=trace, d=d, 
                             D=D, max_order = max_order, maxiter = maxiter, 
                             error_action=error_action, suppress_warnings=suppress_warnings, 
                             stepwise=stepwise, random_state=random_state, n_fits=n_fits, n_jobs = n_jobs)


    try:
        model.fit(ts_data_train)
        if debug:
            print(model_fit.params)
    except ValueError as e:
        print('Value Error while fitting the model Auto ARIMA:', str(e))
        return pd.DataFrame()

    results_df = ts_data_test.to_frame()
    
    try:
        results_df['Issues_FC'] = model.predict(len(ts_data_test))
    except ValueError as e:
        print('Value Error while prediction in Auto ARIMA:', str(e))
        return pd.DataFrame()
        
    if is_bc_:
        results_df['Issues'] = np.exp(results_df['Issues'])
        results_df['Issues_FC'] = np.exp(results_df['Issues_FC'])

    return results_df

In [42]:
def plot_calc_error(method, ts_data_train, results_df):
    if len(results_df) == 0:
        print('Results Dataframe is empty')
        return
    if debug:
        plot_results(method, ts_data_train, results_df)
    calc_error(method,results_df['Issues'],results_df['Issues_FC'])

#bc to indicate whether to boxcox transformation -- to indicate the model
def run_models(ts_data, bc, select='mape', run_all_models=None, run_Forecast_model=None):
    global results_df
    #Train test split - 80:20
    ts_len = len(ts_data)
    train_len = int(ts_len*.8)
    ts_data_train = ts_data[:train_len]
    ts_data_test = ts_data[train_len:]
    
    if bc:
        ts_data_bc = pd.Series(boxcox(ts_data.interpolate(), lmbda=0),name='Issues')
        #Index with all months from first to last
        idx = pd.date_range(ts_data.index.min(), ts_data.index.max(),freq='MS')
        #Reset index for boxcox data set
        ts_data_bc.index = idx

        ts_data_train_bc = ts_data_bc[:train_len]
        ts_data_test_bc = ts_data_bc[train_len:]
    
    cons_res_df['Actual'] = ts_data_test
    if run_all_models==True or run_Forecast_model =='SES':
        method = 'SES'
        results_df = run_simple_exp_smooth(ts_data_train, ts_data_test)
        print("---SES----")
        plot_calc_error(method, ts_data_train, results_df)
    
    if run_all_models==True or run_Forecast_model =='HoltsTrend':
        method = 'HoltsTrend'
        results_df = run_holt_trend(ts_data_train, ts_data_test)
        
        print("---HoltsTrend---")
        plot_calc_error(method, ts_data_train, results_df)
    
    if run_all_models==True or run_Forecast_model =='HWAdditive':
        method = 'HWAdditive'
        results_df = run_holt_wint(ts_data_train, ts_data_test)
        print("---HWAdditive-")
        
        plot_calc_error(method, ts_data_train, results_df)
    
    if run_all_models==True or run_Forecast_model in 'HWMult':
        if bc:
            method = 'HWMult'
            results_df = run_holt_wint(ts_data_train, ts_data_test,seas_='mul')
            print("---HWMult----")
            plot_calc_error(method, ts_data_train, results_df)
            
    if run_all_models==True or run_Forecast_model == 'ARIMA_(1, 1, 1)':

        ord = (1,1,1)
        method = 'ARIMA_'+str(ord)
        results_df = run_arima(ts_data_train, ts_data_test, order_=ord)
        
        print("---ARIMA_(1,1,1)----")
        
        plot_calc_error(method, ts_data_train, results_df)

    
    if bc:
        if run_all_models==True or run_Forecast_model == "ARIMA_BC_(1, 1, 1)":
            ord = (1,1,1)
            method = 'ARIMA_BC_'+str(ord)
            results_df = run_arima(ts_data_train_bc, ts_data_test_bc, is_bc_=True, order_=ord) #TODO - pass Box Cox data
            print("---ARIMA_BC_(1,1,1)----")
            plot_calc_error(method, ts_data_train, results_df)

        if run_all_models==True or run_Forecast_model == "SARIMA_BC_(1, 1, 1)_(1, 1, 1, 12)":
            ord_1 = (1,1,1)
            ord_2 = (1,1,1,12)
            method = 'SARIMA_BC_'+str(ord_1)+'_'+str(ord_2)
            results_df = run_sarimax(ts_data_train_bc, ts_data_test_bc, is_bc_=True, order_=ord_1, seasonal_order_=ord_2)#TODO
            print("---SARIMA_BC_(1,1,1)_(1,1,1,12)----")
            plot_calc_error(method, ts_data_train, results_df)

        if run_all_models==True or run_Forecast_model == "SARIMA_BC_(1, 2, 1)_(1, 1, 1, 12)":
            ord_1 = (1,2,1)
            ord_2 = (1,1,1,12)
            method = 'SARIMA_BC_'+str(ord_1)+'_'+str(ord_2)
            results_df = run_sarimax(ts_data_train_bc, ts_data_test_bc, is_bc_=True, order_=ord_1, seasonal_order_=ord_2) #TODO
            print("---SARIMA_BC_(1,2,1)_(1,1,1,12)----")
            plot_calc_error(method, ts_data_train, results_df)

        if run_all_models==True or run_Forecast_model == "SARIMA_BC_(1, 2, 1)_(0, 1, 1, 12)":
            ord_1 = (1,2,1)
            ord_2 = (0,1,1,12)
            method = 'SARIMA_BC_'+str(ord_1)+'_'+str(ord_2)
            results_df = run_sarimax(ts_data_train_bc, ts_data_test_bc, is_bc_=True, order_=ord_1, seasonal_order_=ord_2) #TODO
            print("---SARIMA_BC_(1,2,1)_(0,1,1,12)----")
            plot_calc_error(method, ts_data_train, results_df)

        if run_all_models==True or run_Forecast_model == "SARIMA_(1, 2, 1)_(1, 1, 1, 12)":
            ord_1 = (1,2,1)
            ord_2 = (1,1,1,12)
            method = 'SARIMA_'+str(ord_1)+'_'+str(ord_2)
            results_df = run_sarimax(ts_data_train, ts_data_test, order_=ord_1, seasonal_order_=ord_2)
        
            print("---SARIMA_(1,2,1)_(1,1,1,12)----")
            plot_calc_error(method, ts_data_train, results_df)

        if run_all_models==True or run_Forecast_model == "SARIMA_(1, 2, 1)_(0, 1, 1, 12)":
            ord_1 = (1,2,1)
            ord_2 = (0,1,1,12)
            method = 'SARIMA_'+str(ord_1)+'_'+str(ord_2)
            results_df = run_sarimax(ts_data_train, ts_data_test, order_=ord_1, seasonal_order_=ord_2)
        
            print("---SARIMA_(1,2,1)_(0,1,1,12)----")
            plot_calc_error(method, ts_data_train, results_df)

    if run_all_models==True or run_Forecast_model =='PMD_Diff1':
        
        method = 'PMD_Diff1'
        results_df = run_auto_arima(ts_data_train, ts_data_test, is_bc_=False, start_p=1, start_q=1,max_p=8, max_q=8,start_P=0, 
                                 start_Q=0, max_P=8, max_Q=8, m=12, seasonal=True, trace=debug, d=1, D=1, 
                                 error_action='warn', suppress_warnings=True, stepwise=True, random_state=20, n_fits=100)
        
        print("---PMD_Diff1----")
        plot_calc_error(method, ts_data_train, results_df)

    if run_all_models==True or run_Forecast_model =='PMD_Diff2':
        method = 'PMD_Diff2'
        results_df = run_auto_arima(ts_data_train, ts_data_test, is_bc_=False, start_p=1, start_q=1,max_p=8, max_d=3, max_q=8,start_P=0, 
                                 start_Q=0, max_P=8, max_Q=8, m=12, seasonal=True, trace=debug, d=2, 
                                 error_action='warn', suppress_warnings=True, stepwise=True, random_state=20, n_fits=100)
        
        print("---PMD_Diff2----")
        plot_calc_error(method, ts_data_train, results_df)

    if run_all_models==True or run_Forecast_model =='PMD_Diff1_BC':
        if bc:
            method = 'PMD_Diff1_BC'
            results_df = run_auto_arima(ts_data_train_bc, ts_data_test_bc, is_bc_=True, start_p=1, start_q=1,max_p=8, max_q=8,start_P=0, 
                                     start_Q=0, max_P=8, max_Q=8, m=12, seasonal=True, trace=debug, d=1, D=1, 
                                     error_action='warn', suppress_warnings=True, stepwise=True, random_state=20, n_fits=100)
            
            print("---PMD_Diff1_BC----")
            plot_calc_error(method, ts_data_train, results_df)
            
    if not run_all_models :        
        display(results_df)


In [44]:
%%time
#Running all models 
#Forecasting with missng values linear interpolation
debug = False

all_res_intp_dfs = {}
all_err_intp_dfs = {}
i = 0
minErrorModel = []
for row in cons_con_comp[['Language','Component']].itertuples():
    con, comp = row[1], row[2]
    print('Building models for Language:{} and Component:{}'.format(con, comp))
    print('='*100)
    ts_data = me_df[(me_df.Language==con)&(me_df.Component==comp)].groupby('Month')['Issues'].sum()
    
    #Index with all months from first to last
    idx = pd.date_range(ts_data.index.min(), ts_data.index.max(),freq='MS')
    #Reindex time series data to fill values with nan and use interpolation
    ts_data = ts_data.reindex(idx, fill_value=np.nan)
    #Interploate to impute
    ts_data = ts_data.interpolate()
    
    print('TS Data Set shape: ',ts_data.shape)

    #Initiate Consolidated Result and Error dataframes
    cons_err_df = pd.DataFrame()
    cons_res_df = pd.DataFrame()

    run_models(ts_data, bc=True,run_all_models=True)
    
    
    all_res_intp_dfs[(con, comp)] = cons_res_df
    all_err_intp_dfs[(con, comp)] = cons_err_df
    
    display(cons_err_df)
    
    for i in cons_err_df.itertuples():
    #print(i[3])
        if i[3] == cons_err_df['RMSE'].min() :
            #print("mehod---",i[1])
            minErrorModel.append(i[1])

            break
    if debug:
        display(cons_res_df)
        
    #For testing.Run only limited combos
#     i += 1
#     if i==1:
#         break
    
cons_con_comp['Min Error Method Interpolation'] = minErrorModel

Building models for Language:Danish and Component:Appeals
TS Data Set shape:  (66,)
---SES----
---HoltsTrend---
---HWAdditive-
---HWMult----
---ARIMA_(1,1,1)----
---ARIMA_BC_(1,1,1)----
---SARIMA_BC_(1,1,1)_(1,1,1,12)----
---SARIMA_BC_(1,2,1)_(1,1,1,12)----
---SARIMA_BC_(1,2,1)_(0,1,1,12)----
---SARIMA_(1,2,1)_(1,1,1,12)----
---SARIMA_(1,2,1)_(0,1,1,12)----
---PMD_Diff1----
---PMD_Diff2----
---PMD_Diff1_BC----


,Method,MAPE,RMSE,MAE,Over Predictions,Under Predictions
0,SES,62.92,14.60,12.259981,9 - 81.71,5 - 29.11
0,HoltsTrend,93.38,30.74,24.707399,0 - 0,14 - 93.38
0,HWAdditive,45.36,13.59,10.883942,5 - 71.37,9 - 30.91
0,HWMult,49.50,14.29,11.506197,5 - 75.88,9 - 34.85
0,"ARIMA_(1, 1, 1)",112.95,20.55,18.852836,11 - 137.72,3 - 22.11
0,"ARIMA_BC_(1, 1, 1)",52.58,14.63,11.414475,8 - 67.91,6 - 32.13
0,"SARIMA_BC_(1, 1, 1)_(1, 1, 1, 12)",71.52,19.39,14.349953,6 - 123.4,8 - 32.62
0,"SARIMA_BC_(1, 2, 1)_(1, 1, 1, 12)",166.82,43.98,29.998606,12 - 192.07,2 - 15.37
0,"SARIMA_BC_(1, 2, 1)_(0, 1, 1, 12)",219.12,54.52,40.230211,13 - 234.04,1 - 25.17
0,"SARIMA_(1, 2, 1)_(1, 1, 1, 12)",179.24,35.66,30.863150,13 - 192.55,1 - 6.18


Building models for Language:Danish and Component:Complaints
TS Data Set shape:  (65,)
---SES----
---HoltsTrend---
---HWAdditive-
---HWMult----
---ARIMA_(1,1,1)----
---ARIMA_BC_(1,1,1)----
---SARIMA_BC_(1,1,1)_(1,1,1,12)----
---SARIMA_BC_(1,2,1)_(1,1,1,12)----
---SARIMA_BC_(1,2,1)_(0,1,1,12)----


C:\Users\v-sivapaturi\Anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


---SARIMA_(1,2,1)_(1,1,1,12)----
---SARIMA_(1,2,1)_(0,1,1,12)----
---PMD_Diff1----
---PMD_Diff2----
---PMD_Diff1_BC----


...

---HWMult----
---ARIMA_(1,1,1)----
---ARIMA_BC_(1,1,1)----
---SARIMA_BC_(1,1,1)_(1,1,1,12)----
---SARIMA_BC_(1,2,1)_(1,1,1,12)----
---SARIMA_BC_(1,2,1)_(0,1,1,12)----
---SARIMA_(1,2,1)_(1,1,1,12)----
---SARIMA_(1,2,1)_(0,1,1,12)----
---PMD_Diff1----
---PMD_Diff2----
---PMD_Diff1_BC----


,Method,MAPE,RMSE,MAE,Over Predictions,Under Predictions
0,SES,170.88,26.92,20.482087,10 - 200.05,3 - 73.64
0,HoltsTrend,336.97,27.01,26.150054,10 - 424.97,3 - 43.64
0,HWAdditive,199.50,26.31,22.026967,10 - 239.33,3 - 66.72
0,HWMult,120.26,37.66,24.661585,3 - 205.24,10 - 94.76
0,"ARIMA_(1, 1, 1)",198.85,25.99,21.318340,10 - 238.23,3 - 67.57
0,"ARIMA_BC_(1, 1, 1)",219.45,25.06,21.661415,10 - 267.01,3 - 60.93
0,"SARIMA_BC_(1, 1, 1)_(1, 1, 1, 12)",191.86,21.82,18.729113,10 - 234.86,3 - 48.55
0,"SARIMA_BC_(1, 2, 1)_(1, 1, 1, 12)",284.08,26.40,21.172617,11 - 330.94,2 - 26.32
0,"SARIMA_BC_(1, 2, 1)_(0, 1, 1, 12)",314.38,30.06,22.558453,11 - 369.15,2 - 13.16
0,"SARIMA_(1, 2, 1)_(1, 1, 1, 12)",300.81,35.78,30.406721,10 - 371.9,3 - 63.81


Building models for Language:Traditional Chinese and Component:Tscore
TS Data Set shape:  (66,)
---SES----
---HoltsTrend---
---HWAdditive-
---HWMult----
---ARIMA_(1,1,1)----
---ARIMA_BC_(1,1,1)----
---SARIMA_BC_(1,1,1)_(1,1,1,12)----
---SARIMA_BC_(1,2,1)_(1,1,1,12)----
---SARIMA_BC_(1,2,1)_(0,1,1,12)----
---SARIMA_(1,2,1)_(1,1,1,12)----
---SARIMA_(1,2,1)_(0,1,1,12)----
---PMD_Diff1----
---PMD_Diff2----
---PMD_Diff1_BC----


,Method,MAPE,RMSE,MAE,Over Predictions,Under Predictions
0,SES,86.82,3.53,3.054917,11 - 102.79,3 - 28.27
0,HoltsTrend,57.88,3.94,3.098146,5 - 84.57,9 - 43.06
0,HWAdditive,217.39,10.52,7.414683,13 - 233.38,1 - 9.55
0,HWMult,283.82,16.99,9.901672,13 - 305.21,1 - 5.8
0,"ARIMA_(1, 1, 1)",168.07,6.42,5.924716,13 - 179.98,1 - 13.24
0,"ARIMA_BC_(1, 1, 1)",118.39,4.58,4.058785,13 - 125.13,1 - 30.73
0,"SARIMA_BC_(1, 1, 1)_(1, 1, 1, 12)",112.62,7.01,4.361514,8 - 175.56,6 - 28.69
0,"SARIMA_BC_(1, 2, 1)_(1, 1, 1, 12)",101.03,5.76,4.120697,6 - 192.77,8 - 32.23
0,"SARIMA_BC_(1, 2, 1)_(0, 1, 1, 12)",105.20,5.96,4.189978,6 - 204.87,8 - 30.45
0,"SARIMA_(1, 2, 1)_(1, 1, 1, 12)",266.05,13.49,12.636714,1 - 453.61,13 - 251.62


Wall time: 22min 53s


In [45]:
cons_con_comp

,Language,Component,FirstMonth,LastMonth,Total,Missing,LastCont,AvgIssues,MissingPerc,DataAvl,Min Error Method Interpolation
18,Danish,Appeals,January2016,June2021,66,0,65,15.136364,0.0,66,HWAdditive
19,Danish,Complaints,January2016,May2021,65,1,53,21.140625,2.0,64,PMD_Diff2
16671,Danish,Tscore,January2016,June2021,66,31,2,1.457143,47.0,35,SES
6893,Dutch,Appeals,January2016,June2021,66,0,65,46.484848,0.0,66,HWAdditive
1174,Dutch,Complaints,January2016,June2021,66,0,65,50.878788,0.0,66,HWMult
8422,Dutch,Enablements,April2019,June2021,27,3,19,5.041667,11.0,24,SES
24716,Dutch,Tscore,February2016,June2021,65,11,13,7.000000,17.0,54,HoltsTrend
16697,English,Accounts & Billing,January2016,June2021,66,0,65,486.712121,0.0,66,"ARIMA_(1, 1, 1)"
4,English,Appeals,January2016,June2021,66,0,65,1626.045455,0.0,66,HoltsTrend
16717,English,Bing Places,January2016,June2021,66,0,65,755.712121,0.0,66,SES


In [46]:


%%time
#Forecasting with missng values linear interpolation
#Forecasting with only the min error model only
debug = False

all_res_intp_dfs = {}
all_err_intp_dfs = {}
i = 0
for row in cons_con_comp[['Language','Component', 'Min Error Method Interpolation']].itertuples():
    con, comp = row[1], row[2]
    print('Building models for Language:{} and Component:{}'.format(con, comp))
    print('='*100)
    ts_data = me_df[(me_df.Language==con)&(me_df.Component==comp)].groupby('Month')['Issues'].sum()
    
    #Index with all months from first to last
    idx = pd.date_range(ts_data.index.min(), ts_data.index.max(),freq='MS')
    #Reindex time series data to fill values with nan and use interpolation
    ts_data = ts_data.reindex(idx, fill_value=np.nan)
    #Interploate to impute
    ts_data = ts_data.interpolate()
    
    print('TS Data Set shape: ',ts_data.shape)

    #Initiate Consolidated Result and Error dataframes
    cons_err_df = pd.DataFrame()
    cons_res_df = pd.DataFrame()

    run_models(ts_data, bc=True,run_all_models=False,run_Forecast_model = row[3])
    
    
    all_res_intp_dfs[(con, comp)] = cons_res_df
    all_err_intp_dfs[(con, comp)] = cons_err_df
    
    display(cons_err_df)
    if debug:
        display(cons_res_df)
        
    #For testing.Run only limited combos
#     i += 1
#     if i==1:
#         break
    
    print()

Building models for Language:Danish and Component:Appeals
TS Data Set shape:  (66,)
---HWAdditive-


,Issues,Issues_FC
2020-05-01,40,14.124963
2020-06-01,19,18.124963
2020-07-01,13,16.124963
2020-08-01,29,21.124963
2020-09-01,21,18.124963
2020-10-01,10,19.124963
2020-11-01,25,19.124963
2020-12-01,12,10.124963
2021-01-01,62,41.124963
2021-02-01,15,23.124963


,Method,MAPE,RMSE,MAE,Over Predictions,Under Predictions
0,HWAdditive,45.36,13.59,10.883942,5 - 71.37,9 - 30.91



Building models for Language:Danish and Component:Complaints
TS Data Set shape:  (65,)
---PMD_Diff2----


,Issues,Issues_FC
2020-05-01,32.0,26.431930
2020-06-01,22.0,28.912210
2020-07-01,27.0,30.677612
2020-08-01,35.0,25.253958
2020-09-01,3.0,24.082259
2020-10-01,25.0,45.044899
2020-11-01,44.0,28.607478
2020-12-01,32.0,31.110340
2021-01-01,23.0,30.027890
2021-02-01,29.0,17.002969


...

,Method,MAPE,RMSE,MAE,Over Predictions,Under Predictions
0,"SARIMA_BC_(1, 1, 1)_(1, 1, 1, 12)",191.86,21.82,18.729113,10 - 234.86,3 - 48.55



Building models for Language:Traditional Chinese and Component:Tscore
TS Data Set shape:  (66,)
---SES----


,Issues,Issues_FC
2020-05-01,4.0,7.471106
2020-06-01,3.0,7.471106
2020-07-01,2.0,7.471106
2020-08-01,6.0,7.471106
2020-09-01,5.0,7.471106
2020-10-01,13.0,7.471106
2020-11-01,6.0,7.471106
2020-12-01,7.0,7.471106
2021-01-01,2.0,7.471106
2021-02-01,3.0,7.471106


,Method,MAPE,RMSE,MAE,Over Predictions,Under Predictions
0,SES,86.82,3.53,3.054917,11 - 102.79,3 - 28.27



Wall time: 33.2 s


In [47]:
end = time.ctime()
begin, end

('Tue Sep 28 20:05:30 2021', 'Tue Sep 28 20:41:16 2021')